In [ ]:
#requirements for script python
pip install pandas sqlite3


1136.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import sqlite3
import json
from datetime import datetime


# Remplacer les valeurs vides dans une colonne par des listes vides
def replace_empty_with_default(lst):
    return lst if lst is not None else []
################################################################################################################
# Initialisation d'un dictionnaire pour stocker les relations par ASN
asn_relations = {}

# Ouvrir le fichier et lire les lignes
with open('../manrs_08_2023/20230601.as-rel.txt', 'r') as file:
    relationships = file.readlines()

# Parcourir chaque ligne dans les relations
for entry in relationships:
    if entry.startswith("# step 1: set peering in clique"):
        continue
    if entry.startswith("# step 2: initial provider assignment"):
        continue
    if entry.startswith('# source:topology'):
        continue
    entry = entry.strip().split('|')
    if len(entry) == 1:
        # Traitement pour une seule colonne
        continue
    asn = entry[1]
    data = entry[0]
    
    
    if asn not in asn_relations:
        # Si c'est la première fois que nous rencontrons cet ASN, initialiser ses relations
        asn_relations[asn] = {
            'asn': asn,
            'providers': [],
            'customers': []
        }
    
        if entry[2] == '0':
            # Si le data est 0, ajouter le data à la liste des customers
            asn_relations[asn]['customers'].append(data)
        elif entry[2] == '-1':
            asn_relations[asn]['providers'].append(data)
    else:
        if entry[2] == '0':
            # Si le data est 0, ajouter le data à la liste des customers
            asn_relations[asn]['customers'].append(data)
        elif entry[2] == '-1':
            asn_relations[asn]['providers'].append(data)
            
# Convertir le dictionnaire en DataFrame
df_relations = pd.DataFrame(list(asn_relations.values()))
##########################################################################################################

###########################################################################################################
# Read nro-delegated-stats data
with open('../manrs_08_2023/nro-delegated-stats', 'r') as file:
    nro_data_brut = file.readlines()
    
# Read ii.as-org.v01.2023-01.json data
with open('../manrs_08_2023/ii.as-org.v01.2023-01.json', 'r') as file:
    asorg_data = json.load(file)
    
# Process nro-delegated-stats data
results = []
nro_datas = []
org_datas = []
id_counter = 1  # Initialize a counter for generating ids

# Process lines
for line in nro_data_brut:
    if 'asn' in line and not '*' in  line:
        nro_datas.append(line.strip().split('|'))
    



    
# Process nro-delegated-stats data
for data in nro_datas:
    if data[2] == 'asn':
        if asorg_data.get(data[3]):
            result = [id_counter, data[0], data[1], data[3], data[5], asorg_data[data[3]]['Sibling ASNs'], asorg_data[data[3]]['Name'], asorg_data[data[3]]['Website']]
            results.append(result)
            id_counter += 1  # Increment the id counter
        else:
            result = [id_counter, data[0], data[1], data[3], data[5], [], "", ""]
            results.append(result)
            id_counter += 1  # Increment the id counter

# Create the first DataFrame
header = ['id', 'registry', 'cc', 'asn', 'date', 'sibling_asns', 'name', 'website']
asninfo_df = pd.DataFrame(results, columns=header)

############################################################################################################

##################################################################################################################
category_df = pd.read_csv('../manrs_08_2023/2023-05_categorized_ases.csv', dtype=str)
selected_columns = ['ASN', 'Category 1 - Layer 1', 'Category 1 - Layer 2']
category_df = category_df.loc[:, selected_columns]

#category_df = category_df[['ASN', 'Category 1 - Layer 1', 'Category 1 - Layer 2']]
header_mapping = {'ASN': 'asn',
                  'Category 1 - Layer 1': 'category_1',
                  'Category 1 - Layer 2': 'category_2'}
category_df = category_df.rename(columns=header_mapping)
##########################################################################################################################

###############################################################################################################################
# Read rovista overview.json data
with open('../manrs_08_2023/overview.json', 'r') as file:
    overview_data = json.load(file)

# Create a list to store extracted data
rovs_data = []

# Loop through the overview data
for value in overview_data:
    asn_parts = value['asn'].split(">")
    if len(asn_parts) >= 2:
        extracted_asn = asn_parts[1].split("<")[0]
        ratio_rov = value['ratio']
        rovs_data.append({'asn': extracted_asn, 'ratio_rov': ratio_rov})

 # Create a DataFrame from the extracted data
ratio_df = pd.DataFrame(rovs_data, columns=['asn', 'ratio_rov'])
#########################################################################################################################################


#Mergin Process
category_df['asn'] = category_df['asn'].str.replace('AS', '')
# Merge DataFrames
merged_df = asninfo_df.merge(category_df, how='left', on='asn')
merged_df = merged_df.merge(df_relations, how='left', on='asn')
merged_df = merged_df.merge(ratio_df, how='left', on='asn')
merged_df['asn'] = merged_df['asn'].str.replace('AS', '')
merged_df['created_at'] = datetime.now().isoformat()
merged_df = merged_df.fillna('')
# Convert the 'sibling_asns', 'provider', and 'peering' columns to strings in the DataFrame
merged_df['sibling_asns'] = merged_df['sibling_asns'].astype(str)
merged_df['providers'] = merged_df['providers'].astype(str)
merged_df['customers'] = merged_df['customers'].astype(str)

# Save the merged DataFrame to a CSV file

merged_df.to_csv('database.csv', index=False)

# Create an SQLite database and save the merged DataFrame as a table
conn = sqlite3.connect('database.db')
merged_df.to_sql('manrs', conn, index=False, if_exists='replace')
conn.close()
